In [4]:
import logging
from large_gcs.graph_generators.contact_graph_generator import (
    ContactGraphGeneratorParams,
    ContactGraphGenerator,
)
from large_gcs.graph.incremental_contact_graph import IncrementalContactGraph
from large_gcs.graph.contact_graph import ContactGraph
from large_gcs.algorithms.search_algorithm import ReexploreLevel
from large_gcs.algorithms.gcs_astar import GcsAstar
from large_gcs.algorithms.gcs_astar_convex_restriction import GcsAstarConvexRestriction
from large_gcs.graph.contact_cost_constraint_factory import (
    contact_shortcut_edge_cost_factory_under,
    contact_shortcut_edge_cost_factory_over_obj_weighted,
)
from large_gcs.cost_estimators.shortcut_edge_ce import ShortcutEdgeCE
from large_gcs.contact.contact_regions_set import ContactRegionParams

import numpy as np
from copy import deepcopy
from IPython.display import HTML


np.set_printoptions(formatter={"float": lambda x: "{0:0.3f}".format(x)})
logging.basicConfig(level=logging.INFO)
# logging.getLogger().setLevel(logging.INFO)
logging.getLogger("drake").setLevel(logging.WARNING)

In [12]:
base_name = "cg_trichal4"

In [13]:
cg = ContactGraph.load_from_file(ContactGraphGeneratorParams.graph_file_path_from_name(base_name))

INFO:large_gcs.graph.contact_graph:Generating contact pair modes for 3 body pairs...
100%|██████████| 3/3 [00:00<00:00, 73.55it/s]
INFO:large_gcs.graph.contact_graph:Loading 20 contact pair modes for 3 bodies...
INFO:large_gcs.graph.contact_graph:Generating contact sets for 179 sets...
100%|██████████| 179/179 [00:00<00:00, 395.65it/s]
INFO:large_gcs.graph.contact_graph:Creating vertex costs...
100%|██████████| 181/181 [00:00<00:00, 12247.43it/s]
INFO:large_gcs.graph.graph:Adding 181 vertices to graph...
100%|██████████| 181/181 [00:00<00:00, 11589.31it/s]
INFO:large_gcs.graph.contact_graph:Creating edge costs...
100%|██████████| 6464/6464 [00:00<00:00, 8216.29it/s]
INFO:large_gcs.graph.contact_graph:Creating edge constraints...
100%|██████████| 6464/6464 [00:01<00:00, 5539.13it/s]
INFO:large_gcs.graph.graph:Adding 6464 edges to graph...
100%|██████████| 6464/6464 [00:01<00:00, 4545.97it/s]
INFO:large_gcs.graph.contact_graph:Created contact graph: GraphParams(dim_bounds=(4, 18), n_vert

In [15]:
cg_inc = IncrementalContactGraph.load_from_file(ContactGraphGeneratorParams.inc_graph_file_path_from_name(base_name))

INFO:large_gcs.graph.contact_graph:Creating vertex costs...
100%|██████████| 2/2 [00:00<00:00, 50840.05it/s]
INFO:large_gcs.graph.graph:Adding 2 vertices to graph...
100%|██████████| 2/2 [00:00<00:00, 24105.20it/s]
INFO:large_gcs.graph.contact_graph:Generating contact pair modes for 3 body pairs...
100%|██████████| 3/3 [00:00<00:00, 56.83it/s]
INFO:large_gcs.graph.incremental_contact_graph:Calculating adjacent contact pair modes (67)
100%|██████████| 67/67 [00:00<00:00, 3827.13it/s]


In [16]:
cg_inc._modes_w_possible_edge_to_target

{'IC|obs0_f2-obj0_f1', 'NC|obs0_f2-obj0_f1'}

In [17]:
cg.incoming_edges("target")

[Edge(u="('IC|obs0_f2-obj0_f1', 'NC|obs0_f1-rob0_f2', 'NC|obj0_f1-rob0_f1')", v='target', costs=[<pydrake.solvers.LinearCost object at 0x7f34858ba1b0>], constraints=[<pydrake.solvers.LinearEqualityConstraint object at 0x7f348579f570>], gcs_edge=<pydrake.geometry.optimization.GraphOfConvexSets.Edge object at 0x7f34856c1030>),
 Edge(u="('IC|obs0_f2-obj0_f1', 'NC|obs0_f1-rob0_f2', 'NC|obj0_f2-rob0_f2')", v='target', costs=[<pydrake.solvers.LinearCost object at 0x7f34858bc530>], constraints=[<pydrake.solvers.LinearEqualityConstraint object at 0x7f34857a1cf0>], gcs_edge=<pydrake.geometry.optimization.GraphOfConvexSets.Edge object at 0x7f34856496f0>),
 Edge(u="('IC|obs0_f2-obj0_f1', 'NC|obs0_f1-rob0_f2', 'NC|obj0_f3-rob0_v0')", v='target', costs=[<pydrake.solvers.LinearCost object at 0x7f34858be470>], constraints=[<pydrake.solvers.LinearEqualityConstraint object at 0x7f34857a4c70>], gcs_edge=<pydrake.geometry.optimization.GraphOfConvexSets.Edge object at 0x7f348564bf70>),
 Edge(u="('IC|obs0_

In [10]:
cg_inc.outgoing_edges("source")

[Edge(u='source', v="('NC|obs0_f0-obj0_v3', 'NC|obs0_f2-rob0_v0', 'NC|obj0_f3-rob0_v0')", costs=[<pydrake.solvers.LinearCost object at 0x7f34443d3a30>], constraints=[<pydrake.solvers.LinearEqualityConstraint object at 0x7f34443d3b70>], gcs_edge=None)]

In [11]:
cg.outgoing_edges("source")

[Edge(u='source', v="('NC|obs0_f0-obj0_v3', 'NC|obs0_f2-rob0_v0', 'NC|obj0_f3-rob0_v0')", costs=[<pydrake.solvers.LinearCost object at 0x7f3440552b70>], constraints=[<pydrake.solvers.LinearEqualityConstraint object at 0x7f34404b6d70>], gcs_edge=<pydrake.geometry.optimization.GraphOfConvexSets.Edge object at 0x7f3440245370>)]

In [ ]:
cg.edge_keys

In [ ]:
cg.plot()

In [ ]:
cg.adj_modes